In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd


In [20]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)  # Predicted probability
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


In [21]:
class SpotifyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(np.argmax(y, axis=1))  # Focal Loss에 맞게 변경

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [22]:
class SpotifyRankPredictor(nn.Module):
    def __init__(self, num_categories):
        super(SpotifyRankPredictor, self).__init__()
        self.input_layer = nn.Linear(8, 64)
        self.block1 = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3)
        )
        self.block2 = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3)
        )
        self.output_layer = nn.Linear(32, num_categories)

    def forward(self, x):
        x = self.input_layer(x)
        x = nn.ReLU()(x)

        identity = x
        x = self.block1(x) + identity

        x = self.block2(x)
        x = self.output_layer(x)
        return x  # Softmax는 CrossEntropyLoss에서 자동 처리


In [23]:
def calculate_class_weights(categories):
    class_counts = np.bincount(categories)
    weights = 1 + np.log1p(np.max(class_counts) / class_counts)
    return torch.FloatTensor(weights)


In [24]:
def preprocess_data(df):
    feature_names = ['Danceability', 'Energy', 'Loudness', 'Speechiness',
                     'Acousticness', 'Liveness', 'Tempo', 'Duration (ms)']
    X = df[feature_names].values

    def rank_to_category(rank):
        if rank <= 30:
            return 0
        elif rank <= 50:
            return 1
        else:
            return 2

    ranks = df['Highest Charting Position'].values
    categories = np.array([rank_to_category(rank) for rank in ranks])

    # 원-핫 인코딩
    num_categories = 3
    y_encoded = np.eye(num_categories)[categories]

    return X, y_encoded, num_categories, feature_names

def apply_smote_and_scale(X, y, scaler):
    # 원-핫 인코딩된 y를 다시 카테고리로 변환
    categories = np.argmax(y, axis=1)

    # 데이터 스케일링
    X_scaled = scaler.transform(X)

    # SMOTE 적용
    smote = SMOTE(random_state=42)
    X_resampled, categories_resampled = smote.fit_resample(X_scaled, categories)

    # 원-핫 인코딩
    y_resampled = np.eye(3)[categories_resampled]

    print("Category counts after SMOTE:", np.bincount(categories_resampled))

    # 클래스 가중치 계산
    class_weights = calculate_class_weights(categories_resampled)

    # 리턴값 추가
    return X_resampled, y_resampled, class_weights

In [25]:
def train_model(model, train_loader, val_loader, criterion, optimizer,
                num_epochs=150, patience=15):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = criterion.to(device)

    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered!")
                break

    return train_losses, val_losses


In [26]:
def analyze_misclassifications(model, test_loader, feature_names, scaler):
    model.eval()
    misclassified_samples = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)

            # y_batch가 이미 정답 레이블인 경우
            actual = y_batch  # 직접 actual로 사용

            # 잘못 예측된 샘플 찾기
            mask = (predicted != actual)
            if mask.any():
                wrong_X = X_batch[mask]
                wrong_pred = predicted[mask]
                wrong_actual = actual[mask]
                wrong_probs = outputs[mask]

                # 원래 스케일로 되돌리기
                original_features = scaler.inverse_transform(wrong_X)

                for i in range(len(wrong_X)):
                    sample = {
                        'actual': wrong_actual[i].item(),
                        'predicted': wrong_pred[i].item(),
                        'confidence': wrong_probs[i].max().item(),
                        'features': dict(zip(feature_names, original_features[i]))
                    }
                    misclassified_samples.append(sample)

    return misclassified_samples

def analyze_feature_importance(model, test_loader, feature_names):
    model.eval()
    feature_impacts = {name: [] for name in feature_names}

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            base_output = model(X_batch)

            # 각 특성에 대해 작은 변화를 주고 출력 변화 관찰
            for i, feature in enumerate(feature_names):
                X_modified = X_batch.clone()
                X_modified[:, i] += 0.1  # 작은 변화
                modified_output = model(X_modified)

                # 출력 변화량 계산
                output_change = torch.abs(modified_output - base_output).mean().item()
                feature_impacts[feature].append(output_change)

    # 평균 영향도 계산
    for feature in feature_names:
        feature_impacts[feature] = np.mean(feature_impacts[feature])

    return feature_impacts

In [27]:
def main():
    # 1. 데이터 로드 및 기본 전처리
    df = pd.read_csv('spotify_dataset.csv')
    X, y_encoded, num_categories, feature_names = preprocess_data(df)

    # 2. 데이터 분할 (SMOTE 적용 전)
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42,
        stratify=y_encoded.argmax(axis=1)
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42,
        stratify=y_temp.argmax(axis=1)
    )

    # 3. StandardScaler 학습 (train set으로만)
    scaler = StandardScaler()
    scaler.fit(X_train)

    # 4. train 데이터에만 SMOTE 적용
    X_train_processed, y_train_processed, class_weights = apply_smote_and_scale(
        X_train, y_train, scaler
    )

    # 5. validation과 test set은 스케일링만 적용
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # 6. 데이터셋 생성
    train_dataset = SpotifyDataset(X_train_processed, y_train_processed)
    val_dataset = SpotifyDataset(X_val_scaled, y_val)
    test_dataset = SpotifyDataset(X_test_scaled, y_test)

    batch_size = 64
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model = SpotifyRankPredictor(num_categories)
    criterion = FocalLoss(alpha=1.0, gamma=2.0)
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

    train_losses, val_losses = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=150,
        patience=15
    )

    # 최고 성능 모델 불러오기
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()

    # Test set evaluation
    test_correct = 0
    test_total = 0
    y_pred = []
    y_true = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(y_batch.numpy())
            y_pred.extend(predicted.numpy())
            test_total += y_batch.size(0)
            test_correct += (predicted == y_batch).sum().item()

    accuracy = 100 * test_correct / test_total
    print(f"\nTest Accuracy: {accuracy:.2f}%")

    conf_matrix = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

    # Additional analysis
    misclassified = analyze_misclassifications(model, test_loader, feature_names, scaler)

    print("\n=== 오분류 패턴 분석 ===")
    for category in range(3):
        wrong_predictions = [s for s in misclassified if s['actual'] == category]
        if wrong_predictions:
            print(f"\n클래스 {category}가 잘못 예측된 경우:")
            for feature in feature_names:
                values = [s['features'][feature] for s in wrong_predictions]
                print(f"{feature}: 평균={np.mean(values):.2f}, 표준편차={np.std(values):.2f}")

    print("\n=== 예측 신뢰도 분석 ===")
    for category in range(3):
        wrong_predictions = [s for s in misclassified if s['actual'] == category]
        if wrong_predictions:
            avg_confidence = np.mean([s['confidence'] for s in wrong_predictions])
            print(f"클래스 {category} 오분류 평균 신뢰도: {avg_confidence:.2f}")

    # Feature importance
    feature_importance = analyze_feature_importance(model, test_loader, feature_names)
    print("\n=== 특성 중요도 분석 ===")
    sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
    for feature, importance in sorted_features:
        print(f"{feature}: {importance:.4f}")

if __name__ == "__main__":
    main()

Category counts after SMOTE: [1102 1102 1102]
Epoch [1/150], Train Loss: 0.5622, Val Loss: 0.4814
Epoch [2/150], Train Loss: 0.4988, Val Loss: 0.4844
Epoch [3/150], Train Loss: 0.4598, Val Loss: 0.4834
Epoch [4/150], Train Loss: 0.4479, Val Loss: 0.5024
Epoch [5/150], Train Loss: 0.4326, Val Loss: 0.4826
Epoch [6/150], Train Loss: 0.4228, Val Loss: 0.4824
Epoch [7/150], Train Loss: 0.4113, Val Loss: 0.4803
Epoch [8/150], Train Loss: 0.4005, Val Loss: 0.4849
Epoch [9/150], Train Loss: 0.3956, Val Loss: 0.4867
Epoch [10/150], Train Loss: 0.3981, Val Loss: 0.4896
Epoch [11/150], Train Loss: 0.3808, Val Loss: 0.5003
Epoch [12/150], Train Loss: 0.3737, Val Loss: 0.4895
Epoch [13/150], Train Loss: 0.3674, Val Loss: 0.4906
Epoch [14/150], Train Loss: 0.3613, Val Loss: 0.4911
Epoch [15/150], Train Loss: 0.3473, Val Loss: 0.5042
Epoch [16/150], Train Loss: 0.3523, Val Loss: 0.4914
Epoch [17/150], Train Loss: 0.3369, Val Loss: 0.5107
Epoch [18/150], Train Loss: 0.3383, Val Loss: 0.5054
Epoch [19

C:\Users\이강민\AppData\Local\Temp\ipykernel_7720\2325631844.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))
